# Projet Green AI : Robot de Tri de Déchets sur Plage 🏖️♻️

**Auteurs :** [Vos Noms]

## 1. Objectif du projet
L'objectif est de développer un système de vision par ordinateur économe en énergie (Green AI) capable de piloter un robot nettoyeur de plage. Le robot doit :
1.  Détecter des déchets (Verre, Plastique, Métal, Papier).
2.  Ignorer les éléments naturels (Coquillages, bois, pierres) pour ne pas perturber l'écosystème.
3.  Estimer sa propre incertitude (Confiance) pour éviter les erreurs de tri critiques.

## 2. Approche Green AI
* **Modèle :** Utilisation de `MobileNetV2` (léger) plutôt que des modèles lourds, pour une inférence embarquée à faible consommation.
* **Données :** Création d'un dataset synthétique ("Augment with Sand") pour réduire le besoin de collecter des milliers de nouvelles photos réelles.
* **Mesure :** Suivi de l'empreinte carbone de l'entraînement via `CodeCarbon`.

In [3]:
import os
import torch
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from PIL import Image
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report

# Import de nos modules locaux (Architecture modulaire)
import sys
sys.path.append('src') # On ajoute le dossier src pour importer nos scripts
from model_definition import build_model
from dataset_loader import get_data_loaders

# Configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Exécution sur : {device}")

Exécution sur : cpu


## 3. Préparation des Données : Le défi du "Domain Gap"

Le dataset d'origine (`RealWaste`) contient des objets sur fond neutre/blanc. Or, notre robot évolue sur du sable.
Si nous entraînons sur fond blanc, le robot échouera sur la plage.

**Solution :** Nous avons développé un script `augment_with_sand.py` qui :
1. Détoure les objets (suppression du fond via IA).
2. Les incruste aléatoirement sur des textures de sable.

In [4]:
# Affichage comparatif : Avant vs Après
def show_comparison(category, img_name):
    # Chemins
    original_path = os.path.join("dataset", "RealWaste", category, img_name)
    sand_path = os.path.join("dataset_sand", category, img_name) # Même nom, dossier différent

    if os.path.exists(original_path) and os.path.exists(sand_path):
        fig, ax = plt.subplots(1, 2, figsize=(10, 5))
        ax[0].imshow(Image.open(original_path))
        ax[0].set_title("Original (Fond vide)")
        ax[0].axis('off')
        
        ax[1].imshow(Image.open(sand_path))
        ax[1].set_title("Généré (Sur Sable)")
        ax[1].axis('off')
        plt.show()
    else:
        print("Image non trouvée pour la démo.")

# Exemple avec une image de votre dataset (changez le nom si besoin)
# Cherche une image au hasard pour la démo
import random
cat = "Plastic"
if os.path.exists(f"dataset_sand/{cat}"):
    img = random.choice(os.listdir(f"dataset_sand/{cat}"))
    show_comparison(cat, img)

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import os

# Chemin vers le fichier généré par train.py
csv_path = "Green AI/models/emissions.csv"

if os.path.exists(csv_path):
    # 1. Lecture des données
    df = pd.read_csv(csv_path)
    
    # 2. Calculs
    total_co2 = df['emissions'].sum()
    total_duration = df['duration'].sum()
    
    print(f"ANALYSE GREEN AI")
    print(f"--------------------")
    print(f"Durée totale d'entraînement : {total_duration:.0f} secondes")
    print(f"Émissions CO2 totales     : {total_co2:.6f} kg")
    print(f"Énergie consommée         : {df['energy_consumed'].sum():.6f} kWh")
    
    # 3. Graphique
    plt.figure(figsize=(8, 5))
    df['emissions'].plot(kind='bar', color='green', alpha=0.7)
    plt.title('Émissions de CO2 par session d\'entraînement')
    plt.ylabel('CO2 (kg)')
    plt.xlabel('Session')
    plt.grid(axis='y', linestyle='--', alpha=0.5)
    plt.show()
    
else:
    print(f"Fichier {csv_path} introuvable.")
    print("Assure-toi d'avoir lancé 'python src/train.py' jusqu'au bout.")

Fichier Green AI/models/emissions.csv introuvable.
Assure-toi d'avoir lancé 'python src/train.py' jusqu'au bout.


In [7]:
## CHARGEMENT DU MODÈLE ET DONNÉES DE VALIDATION
print("Chargement des données de validation pour analyse...")
_, val_loader, class_names = get_data_loaders("dataset_sand", batch_size=32)

# Charger le modèle entraîné
model = build_model(len(class_names), pretrained=False)
model.load_state_dict(torch.load("models/best_model_sand.pth", map_location=device))
model.to(device)
model.eval()

# FAIRE LES PRÉDICTIONS
all_preds = []
all_labels = []

print("Calcul des prédictions...")
with torch.no_grad():
    for inputs, labels in val_loader:
        inputs = inputs.to(device)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.numpy())

# CRÉATION DE LA MATRICE
cm = confusion_matrix(all_labels, all_preds)

# AFFICHAGE
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Greens',
            xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Prédictions')
plt.ylabel('Réalité')
plt.title('Matrice de Confusion : Où le robot se trompe-t-il ?')
plt.show()

# Rapport détaillé (Précision par classe)
print(classification_report(all_labels, all_preds, target_names=class_names))

Chargement des données de validation pour analyse...
Chargement des données depuis : dataset_sand
ERREUR: Le dossier dataset_sand n'existe pas. Lance augment_with_sand.py d'abord !


TypeError: object of type 'NoneType' has no len()